In [1]:
# Your Python code here
!pip install transformers accelerate peft bitsandbytes datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
coldstart_path = input("Enter the full path where the coldstart data should be saved (e.g., mydrive/coldstart_data): ")
os.environ["COLDSTART_FOLDER_PATH"] = coldstart_path
coldstart_file_path = os.getenv("COLDSTART_FOLDER_PATH") #  Coldstart file path ffor initial train data

Enter the full path where the coldstart data should be saved (e.g., mydrive/coldstart_data): /content/drive/MyDrive/Data Projects/DeepSeek project/coldstart_dataset


In [4]:
from datasets import load_dataset, DatasetDict, load_dataset, Dataset

# Define file paths based on the coldstart_file_path
train_file = os.path.join(coldstart_file_path, "train.jsonl")
valid_file = os.path.join(coldstart_file_path, "valid.jsonl")
test_file = os.path.join(coldstart_file_path, "test.jsonl")

# checking the dataset format
with open(train_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

{"text": "<|im_start|>user\nWhat is the Fibonacci sequence?<|im_end|>\n<|im_start|>assistant\nA series where each number is the sum of the two preceding ones, typically starting with 0 and 1.<|im_end|>"}
{"text": "<|im_start|>user\nIf a car travels 60 miles in 2 hours, what is its average speed?<|im_end|>\n<|im_start|>assistant\n<think>Average speed is distance divided by time: 60 miles ÷ 2 hours = 30 miles per hour.</think>\n<answer>30 miles per hour.</answer><|im_end|>"}
{"text": "<|im_start|>user\nFor the arithmetic problem 41 - 14 * 5, what's the answer?<|im_end|>\n<|im_start|>assistant\n<think> \nProblem: For the arithmetic problem 41 - 14 * 5, what's the answer?  \n\nReasoning:  \n\n1. Begin by recognizing that this is a problem involving order of operations (PEMDAS), where multiplication should be performed before subtraction.\n   \n2. Perform the multiplication operation first:\n   14 * 5 = 70\n\n3. Now, rewrite the original expression with this result:\n   41 - 70\n\n4. Next, 

In [5]:
# checking the dataset format
with open(valid_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

{"text": "<|im_start|>user\nHi, how are you?<|im_end|>\n<|im_start|>assistant\nHello! I\u2019m doing well, thanks for asking. How can I help you today?<|im_end|>"}
{"text": "<|im_start|>user\nWrite a poem about the forest.<|im_end|>\n<|im_start|>assistant\nThe forest whispers in emerald hues,\nA secret realm beneath morning dews,\nSunlit leaves and hidden streams,\nA timeless world of waking dreams.<|im_end|>"}
{"text": "<|im_start|>user\nWhat\u2019s the capital of Germany?<|im_end|>\n<|im_start|>assistant\nThe capital of Germany is Berlin.<|im_end|>"}
{"text": "<|im_start|>user\nSuggest a good self-improvement book.<|im_end|>\n<|im_start|>assistant\nYou might enjoy \"Atomic Habits\" by James Clear\u2014great for building and breaking habits effectively.<|im_end|>"}
{"text": "<|im_start|>user\nHow do I boil an egg perfectly?<|im_end|>\n<|im_start|>assistant\nPlace eggs in cold water, bring to a rolling boil, then turn off the heat and cover. After 8\u201310 minutes, transfer them to ic

In [6]:
# checking the dataset format
with open(test_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

{"text": "<|im_start|>user\nTranslate this sentence into French: 'I enjoy learning new skills.'<|im_end|>\n<|im_start|>assistant\nJ\u2019aime apprendre de nouvelles comp\u00e9tences.<|im_end|>"}
{"text": "<|im_start|>user\nGive me a random historical fact.<|im_end|>\n<|im_start|>assistant\nThe Great Wall of China isn\u2019t a single continuous wall, but a series of fortifications built over centuries.<|im_end|>"}
{"text": "<|im_start|>user\nTell me a joke about cats.<|im_end|>\n<|im_start|>assistant\nSure. Why did the cat join the Red Cross? Because it wanted to be a first-aid kit-ten!<|im_end|>"}
{"text": "<|im_start|>user\nWhat is the best way to cook a steak?<|im_end|>\n<|im_start|>assistant\nFor a perfect medium-rare steak, sear in a hot pan for 30 seconds on each side, then bake in a 400°F oven for 2 minutes.<|im_end|>"}
{"text": "<|im_start|>user\nWhat is the most popular sport in Japan?<|im_end|>\n<|im_start|>assistant\nBaseball is the most popular sport in Japan, followed by so

In [8]:
import json

# Function to load JSONL file into a Hugging Face dataset
def load_jsonl_as_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = [json.loads(line.strip()) for line in file]  # Load all lines efficiently
    return Dataset.from_list(data)


In [20]:
dataset_train = load_jsonl_as_dataset(train_file)
dataset_valid = load_jsonl_as_dataset(valid_file)
dataset_test = load_jsonl_as_dataset(test_file)

In [21]:
# Print dataset structure
print(dataset_train, "\n", dataset_valid, "\n", dataset_test)

Dataset({
    features: ['text'],
    num_rows: 382
}) 
 Dataset({
    features: ['text'],
    num_rows: 14
}) 
 Dataset({
    features: ['text'],
    num_rows: 26
})


In [22]:
type(dataset_train)

datasets.arrow_dataset.Dataset

In [23]:
print(dataset_valid["text"][0])

<|im_start|>user
Hi, how are you?<|im_end|>
<|im_start|>assistant
Hello! I’m doing well, thanks for asking. How can I help you today?<|im_end|>


In [24]:
import re

def extract_qa_from_message(message):
    """
    Extracts the user question and assistant answer from a given message.
    """
    # Regex pattern to match <|im_start|>user and <|im_start|>assistant segments
    user_pattern = r"<\|im_start\|>user\s*(.*?)<\|im_end\|>"
    assistant_pattern = r"<\|im_start\|>assistant\s*(.*?)<\|im_end\|>"

    # Extract question and answer using regex
    user_match = re.search(user_pattern, message, re.DOTALL)
    assistant_match = re.search(assistant_pattern, message, re.DOTALL)

    # Get extracted text or None
    question = user_match.group(1).strip() if user_match else None
    answer = assistant_match.group(1).strip() if assistant_match else None

    return question, answer

def create_qa_dataset(messages):
    """
    Processes a list of messages and creates a dataset with 'question' and 'answer' columns.
    """
    questions, answers = [], []

    for message in messages:
        question, answer = extract_qa_from_message(message)
        if question and answer:  # Ensure both exist
            questions.append(question)
            answers.append(answer)

    # Convert to Hugging Face Dataset format
    dataset = Dataset.from_dict({"question": questions, "answer": answers})
    return dataset


In [25]:
# function for creating dataset with question and answer columns

def process_messages_to_qa_dataset(messages):
    """
    Extracts questions and answers from a list of messages and returns a structured dataset.
    """
    questions, answers = [], []

    for message in messages:
        # Regex patterns to extract user and assistant messages
        user_match = re.search(r"<\|im_start\|>user\s*(.*?)<\|im_end\|>", message, re.DOTALL)
        assistant_match = re.search(r"<\|im_start\|>assistant\s*(.*?)<\|im_end\|>", message, re.DOTALL)

        # Extracted content
        question = user_match.group(1).strip() if user_match else None
        answer = assistant_match.group(1).strip() if assistant_match else None

        if question and answer:  # Ensure valid Q&A pairs
            questions.append(question)
            answers.append(answer)

    # Convert extracted data to a Hugging Face Dataset
    return Dataset.from_dict({"question": questions, "answer": answers})


In [26]:
qa_dataset_train = process_messages_to_qa_dataset(dataset_train["text"])
qa_dataset_valid = process_messages_to_qa_dataset(dataset_valid["text"])
qa_dataset_test = process_messages_to_qa_dataset(dataset_test["text"])

In [27]:
print(qa_dataset_train.column_names)
print(qa_dataset_train[1])

['question', 'answer']
{'question': 'If a car travels 60 miles in 2 hours, what is its average speed?', 'answer': '<think>Average speed is distance divided by time: 60 miles ÷ 2 hours = 30 miles per hour.</think>\n<answer>30 miles per hour.</answer>'}


In [28]:
import pandas as pd

# Define the save path
save_path = "/content/drive/MyDrive/Data Projects/DeepSeek project/coldstart_dataset_csv/"

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)

# Convert to DataFrame and save
pd.DataFrame(qa_dataset_train).to_csv(save_path + "coldstart_dataset_train.csv", index=False)
pd.DataFrame(qa_dataset_valid).to_csv(save_path + "coldstart_dataset_valid.csv", index=False)
pd.DataFrame(qa_dataset_test).to_csv(save_path + "coldstart_dataset_test.csv", index=False)

print(f"Datasets saved to {save_path}")


Datasets saved to /content/drive/MyDrive/Data Projects/DeepSeek project/coldstart_dataset_csv/
